In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv('UNSW_NB15_training-set.csv')

In [3]:
df.dtypes

id                     int64
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean                  int64
trans_depth            int64
response_body_len      int64
ct_srv_src             int64
ct_state_ttl           int64
ct_dst_ltm             int64
ct_src_dport_l

In [4]:
#df_1 = pd.get_dummies(df, columns = ['proto','service','state'])
label_encoder = LabelEncoder()
for col in ['proto', 'service', 'state', 'attack_cat']:
    df[col] = label_encoder.fit_transform(df[col])
df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,113,0,2,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,6,0
1,2,0.649902,113,0,2,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,6,0
2,3,1.623129,113,0,2,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,6,0
3,4,1.681642,113,3,2,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,6,0
4,5,0.449454,113,0,2,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,175337,0.000009,119,2,3,2,0,114,0,111111.107200,...,13,24,0,0,0,24,24,0,5,1
175337,175338,0.505762,113,0,2,10,8,620,354,33.612649,...,1,2,0,0,0,1,1,0,8,1
175338,175339,0.000009,119,2,3,2,0,114,0,111111.107200,...,3,13,0,0,0,3,12,0,5,1
175339,175340,0.000009,119,2,3,2,0,114,0,111111.107200,...,14,30,0,0,0,30,30,0,5,1


In [5]:
#data = df_1.drop(columns = ['proto_3pc','proto_icmp','proto_a/n','proto_aes-sp3-d','proto_any','proto_argus','proto_aris','proto_ax.25','proto_bbn-rcc','proto_bna','proto_br-sat-mon','proto_cbt','proto_cftp','proto_chaos','proto_compaq-peer','proto_cphb','proto_cpnx','proto_crtp','proto_crudp','proto_dcn','proto_ddp','proto_ddx','proto_dgp','proto_egp','proto_eigrp','proto_emcon','proto_encap','proto_etherip','proto_fc','proto_fire','proto_ggp','proto_gmtp','proto_gre','proto_hmp','proto_i-nlsp','proto_iatp','proto_ib','proto_idpr','proto_idpr-cmtp','proto_idrp','proto_ifmp','proto_igmp','proto_igp','proto_il','proto_ip','proto_ipcomp','proto_ipcv','proto_ipip','proto_iplt','proto_ipnip','proto_ippc','proto_ipv6','proto_ipv6-frag','proto_ipv6-no','proto_ipv6-opts','proto_ipv6-route','proto_ipx-n-ip','proto_irtp','proto_isis','proto_iso-ip','proto_iso-tp4','proto_kryptolan','proto_l2tp','proto_larp','proto_leaf-1','proto_leaf-2','proto_merit-inp','proto_mfe-nsp','proto_mhrp','proto_micp','proto_mobile','proto_mtp','proto_mux','proto_narp','proto_netblt','proto_nsfnet-igp','proto_nvp','proto_pgm','proto_pim','proto_pipe','proto_pnni','proto_pri-enc','proto_prm','proto_ptp','proto_pup','proto_pvp','proto_qnx','proto_rdp','proto_rsvp','proto_rtp','proto_rvd','proto_sat-expak','proto_sat-mon','proto_sccopmce','proto_scps','proto_sctp','proto_sdrp','proto_secure-vmtp','proto_sep','proto_skip','proto_sm','proto_smp','proto_snp','proto_sprite-rpc','proto_sps','proto_srp','proto_st2','proto_stp','proto_sun-nd','proto_swipe','proto_tcf','proto_tlsp','proto_tp++','proto_trunk-1','proto_trunk-2','proto_ttp','proto_uti','proto_vines','proto_visa','proto_vmtp','proto_vrrp','proto_wb-expak','proto_wb-mon','proto_wsn','proto_xnet','proto_xns-idp','proto_xtp','proto_zero','service_dhcp','service_ftp','service_ftp-data','service_irc','service_pop3','service_radius','service_snmp','service_ssh','service_ssl','state_CON','state_ECO','state_PAR','state_RST','state_URN','state_no'])
#data_1 = df.drop(columns = ['attack_cat'])
corr = df.corr()
corr.style.background_gradient(cmap = "coolwarm")

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,sjit,djit,swin,stcpb,dtcpb,dwin,tcprtt,synack,ackdat,smean,dmean,trans_depth,response_body_len,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,1.000000,0.006587,0.013943,0.038238,0.518446,-0.068827,-0.126316,0.001104,-0.080283,0.364575,0.615508,-0.049604,0.143649,-0.356902,-0.018451,-0.099755,0.027197,-0.027327,-0.019626,-0.081799,-0.416529,-0.326110,-0.326044,-0.412318,0.001216,-0.004703,0.007422,-0.083356,-0.361818,-0.019404,-0.020558,0.478979,0.617467,0.467591,0.536544,0.601158,0.539093,-0.037636,-0.037636,-0.028287,0.425396,0.480899,0.033142,-0.271955,0.727173
dur,0.006587,1.000000,-0.124502,-0.008234,0.103443,0.254559,0.181182,0.199731,0.144134,-0.120966,0.012196,0.044159,-0.081749,-0.050603,0.198597,0.142963,0.080055,0.152142,0.144413,0.157443,0.022047,0.013183,0.014724,0.017527,0.053125,0.051093,0.049332,0.090028,0.025336,0.002071,0.078915,-0.113709,0.186293,-0.086300,-0.094091,-0.093923,-0.101760,0.020641,0.020641,0.024743,-0.080871,-0.115336,0.035370,-0.097825,0.036175
proto,0.013943,-0.124502,1.000000,0.170032,-0.172441,0.013469,0.026439,0.005920,0.015812,0.013924,0.049944,0.113184,0.004759,0.046375,0.011392,0.020002,-0.562789,-0.052417,0.016011,0.019388,0.138967,0.108571,0.108630,0.137605,0.079193,0.073528,0.076362,0.042157,0.077296,0.020709,0.006005,0.203057,-0.162433,0.191101,0.174965,0.165796,0.175708,0.018003,0.018003,0.028809,0.168121,0.198594,-0.585941,0.174490,0.020267
service,0.038238,-0.008234,0.170032,1.000000,-0.144978,0.114403,0.077338,0.105188,0.035492,-0.141709,-0.295302,0.262970,-0.166339,-0.099581,0.114522,0.051495,-0.089971,-0.020190,-0.011469,0.090262,0.292887,0.237103,0.237723,0.300035,0.140239,0.110995,0.155811,0.224861,0.145641,0.191839,0.056951,-0.058269,-0.205943,0.047685,0.038347,0.051106,-0.006774,0.071051,0.071051,0.266206,0.028599,-0.048011,-0.088847,-0.056009,0.035052
state,0.518446,0.103443,-0.172441,-0.144978,1.000000,-0.078701,-0.098268,-0.049300,-0.059759,0.432307,0.584697,-0.375533,0.292570,-0.150501,-0.060125,-0.071056,0.095492,-0.076235,-0.045441,-0.064747,-0.367493,-0.314361,-0.313922,-0.397710,-0.278469,-0.261882,-0.264946,-0.070796,-0.256392,-0.056128,-0.025541,0.385515,0.759825,0.328748,0.372309,0.408662,0.429906,-0.051970,-0.051970,-0.078856,0.323019,0.387446,0.094198,-0.237008,0.497685
spkts,-0.068827,0.254559,0.013469,0.114403,-0.078701,1.000000,0.390067,0.963791,0.206609,-0.076358,-0.102723,0.068246,-0.051646,0.075897,0.971069,0.207798,-0.017587,-0.001678,-0.000384,0.017096,0.131813,0.107410,0.102161,0.133102,0.039187,0.035507,0.038725,0.216592,0.150237,0.008834,0.087217,-0.069127,-0.086170,-0.060194,-0.068373,-0.072484,-0.077553,0.009951,0.009951,0.006084,-0.061584,-0.069598,-0.017770,-0.008428,-0.052178
dpkts,-0.126316,0.181182,0.026439,0.077338,-0.098268,0.390067,1.000000,0.188476,0.971907,-0.098202,-0.192580,0.053861,-0.066710,0.139145,0.204883,0.978636,-0.022160,-0.006514,0.000229,0.054371,0.183703,0.144119,0.142667,0.185555,0.020915,0.015936,0.023899,0.014697,0.441445,0.029042,0.442194,-0.079095,-0.150023,-0.071909,-0.086695,-0.094267,-0.094085,0.013491,0.013491,0.047974,-0.075190,-0.078342,-0.021765,0.035163,-0.118591
sbytes,0.001104,0.199731,0.005920,0.105188,-0.049300,0.963791,0.188476,1.000000,0.009926,-0.028468,-0.020860,0.063009,-0.018322,-0.007829,0.996109,0.006804,-0.006565,0.000024,-0.002054,-0.003516,0.050450,0.043164,0.037988,0.050981,0.043624,0.039739,0.042883,0.232348,-0.004973,0.003428,0.001620,-0.034395,-0.012053,-0.026661,-0.026490,-0.027281,-0.032061,-0.004515,-0.004515,-0.002185,-0.027479,-0.034553,-0.006367,-0.040022,0.018576
dbytes,-0.080283,0.144134,0.015812,0.035492,-0.059759,0.206609,0.971907,0.009926,1.000000,-0.059475,-0.135515,0.023559,-0.040430,0.104757,0.017366,0.996504,-0.013618,-0.007701,-0.002422,0.0

In [6]:
from collections import Counter

xs = df.drop(columns = ['id','label', 'attack_cat'])
ys = df[['label']]

X_train = xs
y_train = ys.values.ravel()


smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train, y_train)

#counter = Counter(y_train)

#for k,v in counter.items():
#    per = v / len(y_train) * 100
#    print('Class = %d, n = %d (%.3f%%)'% (k, v, per))

 
#X_train, X_val, y_train, y_val = train_test_split(xs,ys,train_size = 0.7, stratify = ys, random_state = 42)

In [7]:
df_2 = pd.read_csv('UNSW_NB15_testing-set.csv')

for col in ['proto', 'service', 'state', 'attack_cat']:
    df_2[col] = label_encoder.fit_transform(df_2[col])
df_2
test_x = df_2.drop(columns = ['id','label', 'attack_cat'])
test_y = df_2[['label']]

X_test = test_x
y_test = test_y

In [8]:
from sklearn.naive_bayes import BernoulliNB

steps = [('scaler', StandardScaler()), 
         ('NB', BernoulliNB())]

pipe = Pipeline(steps)

pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('NB', BernoulliNB())])

In [9]:

y_train_predict = pipe.predict(X_train)

score = accuracy_score(y_train, y_train_predict)
precision = precision_score(y_train, y_train_predict, average='macro')
recall = recall_score(y_train, y_train_predict, average='macro')
f1 = f1_score(y_train, y_train_predict, average='macro')
print(f"Accuracy: {score:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Accuracy: 0.7955, Precision: 0.8107, Recall: 0.7955, F1 Score: 0.7929


In [10]:

y_test_predict = pipe.predict(X_test)

score = accuracy_score(y_test, y_test_predict)
precision = precision_score(y_test, y_test_predict, average='macro')
recall = recall_score(y_test, y_test_predict, average='macro')
f1 = f1_score(y_test, y_test_predict, average='macro')
print(f"Accuracy: {score:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Accuracy: 0.7179, Precision: 0.7166, Recall: 0.7096, F1 Score: 0.7111
